In [ ]:
import numpy as np
import csv
from keras import optimizers
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, Flatten, concatenate
from keras.layers import Convolution3D, MaxPooling3D, Convolution2D, MaxPooling2D, BatchNormalization,UpSampling2D
from keras.utils import np_utils
from keras.optimizers import Adam
import random
import os
import cv2
import tensorflow as tf
from keras.layers import Lambda

In [ ]:
# PART 2 : FUNCTION

def part_2 (_anchor):
  
  first_convolution = Convolution2D(256, (5, 3), activation='relu', padding='same')(_anchor)
  second_convolution = Convolution2D(256, (3, 1), activation='relu', padding='same')(_anchor)
    
  anchor_concat = concatenate([ first_convolution, second_convolution],-1)
  #anchor_concat = tf.keras.layers.concatenate([ first_convolution, second_convolution],axis=-1)
  
  return anchor_concat;

In [ ]:
# PART 4 : BASE FUNCTION

def part_4_base (_input4):
  flatten_1 = Flatten()(_input4)
  dense_1 = Dense(2048, activation='sigmoid')(flatten_1)
  dropout_1 = Dropout(0.5)(dense_1)
  dense_2 = Dense(2048, activation='sigmoid')(dropout_1)
  dropout_2 = Dropout(0.5)(dense_2)
  
  return dropout_2;

In [ ]:
# PART 4 : PLATE CLASSFICATION FUNCTION

def part_4_plate_classification (base_4):
  plate_classification = Dense(1, activation='sigmoid')(base_4)
  
  return plate_classification;

In [ ]:
# PART 4 : PLATE REGRESSION FUNCTION

def part_4_plate_regression(base_4):
  plate_regression = Dense(4)(base_4)
  
  return plate_regression;

In [ ]:
# PART 1 : LOW LEVEL FEATURE EXTRACTION

input_shape = (1500, 700, 3)
_input = Input(shape=input_shape)

conv_1 = Convolution2D(64, (3, 3), input_shape=input_shape, padding='same', activation='relu')(_input)
conv_2 = Convolution2D(64, (3, 3), activation='relu', padding='same')(conv_1)
conv_3 = Convolution2D(128, (3, 3), activation='relu', padding='same')(conv_2)
conv_4 = Convolution2D(128, (3, 3), activation='relu', padding='same')(conv_3)
conv_5 = Convolution2D(256, (3, 3), activation='relu', padding='same')(conv_4)
conv_6 = Convolution2D(256, (3, 3), activation='relu', padding='same')(conv_5)
conv_7 = Convolution2D(256, (3, 3), activation='relu', padding='same')(conv_6)
conv_8 = Convolution2D(512, (3, 3), activation='relu', padding='same')(conv_7)
conv_9 = Convolution2D(512, (3, 3), activation='relu', padding='same')(conv_8)
conv_10 = Convolution2D(512, (3, 3), activation='relu', padding='same')(conv_9)
maxpool_1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv_10)
conv_11 = Convolution2D(512, (3, 3), activation='relu', padding='same')(maxpool_1)
conv_12 = Convolution2D(512, (3, 3), activation='relu', padding='same')(conv_11)
conv_13 = Convolution2D(256, (3, 3), activation='relu', padding='same')(conv_12)
maxpool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv_13)

_output1 = maxpool_2

In [ ]:
# ANCHORS : PART 2 : PART 4
from tqdm import tqdm
anchors = [(12,12,2,2),(19,20,3,4),(22,22,5,5),(34,35,6,7),(42,42,8,8),(49,50,9,10)]
output = []

for w_l, w_r, h_l, h_r in anchors:
  for i in tqdm(range( w_l, 375-w_r, 10)):
    for j in range( h_l, 175-h_r, 10):
      
      anchor = _output1[:,i-w_l:i+w_r,j-h_l:j+h_r,:]
      filter_concat = part_2(anchor)
      
      base_4 = part_4_base(filter_concat)
      plate_classification = part_4_plate_classification(base_4)
      plate_regression = part_4_plate_regression(base_4)
      
      x = i + ( tf.reshape(plate_regression[:,0],[-1,1]) * ( (w_r*2) + 1))
      y = j + ( tf.reshape(plate_regression[:,1],[-1,1]) * ( (h_r*2) + 1))
      w = ((h_r*2)+1) * tf.exp(tf.reshape(plate_regression[:,2],[-1,1]))
      h = ((h_r*2)+1) * tf.exp(tf.reshape(plate_regression[:,3],[-1,1]))
      p = plate_classification
      
      result = concatenate([x,y,w,h,p])
      
     
      output.append(result)

In [ ]:
model = Model(inputs=[_input], outputs=[output])
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')